In [13]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import io
import gradio as gr

# Authenticate client
prediction_key = "00985879e591465d874bf6f503b501b6"
prediction_endpoint = "https://team1customvision-prediction.cognitiveservices.azure.com/"
project_id = "52a79c5c-b304-4134-83a9-2cdffb34734e"
model_name = "Iteration1"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

data = {
    "Inputs": {
        "input1": [
            {
                "paper": "이건 종이야!📄\n종이는 종이끼리 모여진 데 버려주면 돼!",
                "glass": "이건 유리야!🔍\n유리병은 깨질 수 있으니 던지지 말고 조심히 버려줘!",
                "metal": "이건 금속이야!🔧\n날카로운 모서리에 손이 베이지 않게 조심해!",
                "plastic": "이건 플라스틱이야!🌱\n페트병은 라벨을 떼고 찌그러트려 버려줘!",
                "trash": "이건 일반쓰레기야!🗑️\n일반쓰레기는 종량제 봉투에 모아줄래?\n종량제 봉투가 뭐냐구? 함께 알아볼까?"
            }
        ]
    },
    "GlobalParameters": {}
}

def predict_image(input_img):
    h, w = input_img.size
    img_byte_arr = io.BytesIO() 
    input_img.save(img_byte_arr, format='PNG') 
    img_bytes = img_byte_arr.getvalue() 
    results = predictor.detect_image(project_id, model_name, img_bytes) 

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.axis('off') 

    draw = ImageDraw.Draw(input_img) 
    lineWidth = int(w / 100) 
    color = 'magenta'
    
    detected_objects = []
    
    print("Predictions:")
    for prediction in results.predictions:
        print(f"{prediction.tag_name}: {prediction.probability * 100:.2f}%")
        if (prediction.probability * 100) > 50:  # 확률 임계값을 50%로 낮춤
            detected_objects.append(prediction.tag_name)
            left = prediction.bounding_box.left * w 
            top = prediction.bounding_box.top * h 
            width = prediction.bounding_box.width * w 
            height = prediction.bounding_box.height * h 

            points = ((left, top), (left + width, top), (left + width, top + height), (left, top + height), (left, top)) 
            draw.line(points, fill=color, width=lineWidth) 
            ax.annotate(prediction.tag_name + ' {0:.2f}%'.format(prediction.probability * 100), (left, top), color=color)
    
    ax.imshow(input_img)
    
    # 예측 결과 텍스트 생성
    if detected_objects:
        predictions_text = "\n".join(
            data["Inputs"]["input1"][0].get(obj, "무슨 쓰레기인지 잘 모르겠어. 주위를 정리하고 사진을 다시 찍어줄래?") for obj in detected_objects
        )
    else:
        predictions_text = "무슨 쓰레기인지 잘 모르겠어. 주위를 정리하고 사진을 다시 찍어줄래?"
    
    return fig, predictions_text

result_output = gr.Textbox(label="Result")

demo = gr.Interface(
    fn=predict_image, 
    inputs=gr.Image(type='pil'), 
    outputs=[gr.Plot(), result_output]
)
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7904

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Predictions:
glass: 2.24%
metal: 1.28%
metal: 1.24%
paper: 1.69%
paper: 1.07%
plastic: 99.90%
plastic: 2.60%
plastic: 1.31%
trash: 1.13%
trash: 1.06%
Predictions:
metal: 3.55%
metal: 2.02%
metal: 1.44%
paper: 99.89%
paper: 70.64%
paper: 1.45%
plastic: 2.69%
plastic: 1.86%
plastic: 1.14%
trash: 1.22%
Predictions:
glass: 2.24%
metal: 1.28%
metal: 1.24%
paper: 1.69%
paper: 1.07%
plastic: 99.90%
plastic: 2.60%
plastic: 1.31%
trash: 1.13%
trash: 1.06%
Predictions:
metal: 98.62%
trash: 13.64%
trash: 1.19%
Predictions:
metal: 3.55%
metal: 2.02%
metal: 1.44%
paper: 99.89%
paper: 70.64%
paper: 1.45%
plastic: 2.69%
plastic: 1.86%
plastic: 1.14%
trash: 1.22%
